In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import tensorflow_datasets as tfds

In [2]:
import urllib3
urllib3.disable_warnings()

#tfds.disable_progress_bar()   # 이 주석을 풀면 데이터셋 다운로드과정의 프로그레스바가 나타나지 않습니다.

(ds_train, ds_test), ds_info = tfds.load(
    'cifar10',
    split=['train', 'test'],
    shuffle_files=True,
    with_info=True,
)

In [3]:
def normalize_and_resize_img(image, label):
    """Normalizes images: `uint8` -> `float32`."""
    # image = tf.image.resize(image, [32, 32])
    return tf.cast(image, tf.float32) / 255., label

In [4]:
def apply_normalize_on_dataset(ds, is_test=False, batch_size=16):
    ds = ds.map(
        normalize_and_resize_img, 
        num_parallel_calls=1
    )
    ds = ds.batch(batch_size)
    if not is_test:
        ds = ds.repeat()
        ds = ds.shuffle(200)
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
    return ds

In [5]:
class ConvBlock(tf.keras.Model):
    def __init__(self, channel, strides = 1):
        super(ConvBlock,self).__init__()
        self.conv1 = tf.keras.layers.Conv2D(channel, kernel_size=3, strides=strides, padding='same')
        self.bn1 = tf.keras.layers.BatchNormalization()
        
    def call(self, input_tensor, training = False):
        x = self.conv1(input_tensor)
        x = self.bn1(x, training = training)
        return x

In [6]:
class ResBlock(tf.keras.Model):
    def __init__(self, channel,down_sample = False):
        super(ResBlock,self).__init__()
        self.down_sample = down_sample
        
        if self.down_sample:
            self.conv_block1 = ConvBlock(channel, strides=2)
        else:
            self.conv_block1 = ConvBlock(channel, strides=1)
            
        self.conv_block2 = ConvBlock(channel, strides=1)        
        
        if self.down_sample:
            self.downsample_conv = tf.keras.layers.Conv2D(channel, kernel_size=1, strides=2, padding='same')
            self.downsample_bn = tf.keras.layers.BatchNormalization()
    
    def call(self, input_tensor, training = False):
        x = self.conv_block1(input_tensor)
        x = tf.nn.relu(x)
        x = self.conv_block2(x)
        
        if self.down_sample:
            input_tensor = self.downsample_conv(input_tensor)
            input_tensor = self.downsample_bn(input_tensor, training=training)

        x = x + input_tensor
        x = tf.nn.relu(x)
        return x

In [7]:
class ResNet34(tf.keras.Model):
    def __init__(self):
        super(ResNet34,self).__init__()
        
        self.conv = tf.keras.layers.Conv2D(64, kernel_size=7, strides=2, padding='same')
        self.bn = tf.keras.layers.BatchNormalization()     
        self.maxpool = tf.keras.layers.MaxPool2D(pool_size=2)
               
        self.conv1_1 = ResBlock(64, down_sample = True)
        self.conv1_2 = ResBlock(64)
        self.conv1_3 = ResBlock(64)

        self.conv2_1 = ResBlock(128, down_sample = True)
        self.conv2_2 = ResBlock(128)
        self.conv2_3 = ResBlock(128)      
        self.conv2_4 = ResBlock(128)      
        
        self.conv3_1 = ResBlock(256, down_sample = True)
        self.conv3_2 = ResBlock(256)
        self.conv3_3 = ResBlock(256)      
        self.conv3_4 = ResBlock(256)           
        self.conv3_5 = ResBlock(256)      
        self.conv3_6 = ResBlock(256)      
        
        self.conv4_1 = ResBlock(512, down_sample = True)
        self.conv4_2 = ResBlock(512)
        self.conv4_3 = ResBlock(512)              
        
        self.gap = tf.keras.layers.GlobalAveragePooling2D()
        self.fc1 = tf.keras.layers.Dense(4096, activation='relu')
        self.fc2 = tf.keras.layers.Dense(4096, activation='relu')
        self.fc3 = tf.keras.layers.Dense(10, activation='softmax')        
        
    def call(self, input_tensor, training = False):
        x = self.conv(input_tensor)
        x = self.bn(x)
        x = tf.nn.relu(x)
        x = self.maxpool(x)
        
        x = self.conv1_1(x)
        x = self.conv1_2(x)
        x = self.conv1_3(x)
        
        x = self.conv2_1(x)
        x = self.conv2_2(x)
        x = self.conv2_3(x)
        x = self.conv2_4(x)
        
        x = self.conv3_1(x)
        x = self.conv3_2(x)
        x = self.conv3_3(x)
        x = self.conv3_4(x)
        x = self.conv3_5(x)
        x = self.conv3_6(x)
        
        x = self.conv4_1(x)
        x = self.conv4_2(x)
        x = self.conv4_3(x)
        
        x = self.gap(x)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x

In [8]:
BATCH_SIZE = 256
EPOCH = 20

(ds_train, ds_test), ds_info = tfds.load(
    'cifar10',
    split=['train', 'test'],
    as_supervised=True,
    shuffle_files=True,
    with_info=True,
)
ds_train = apply_normalize_on_dataset(ds_train, batch_size=BATCH_SIZE)
ds_test = apply_normalize_on_dataset(ds_test, batch_size=BATCH_SIZE)

In [9]:
resnet34 = ResNet34()

resnet34.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.SGD(lr=0.01, clipnorm=1.),
    metrics=['accuracy'],
)

history_resnet34 = resnet34.fit(
    ds_train,
    steps_per_epoch=int(ds_info.splits['train'].num_examples/BATCH_SIZE),
    validation_steps=int(ds_info.splits['test'].num_examples/BATCH_SIZE),
    epochs=EPOCH,
    validation_data=ds_test,
    verbose=1
)

Epoch 1/20
195/195 [==============================] - 12s 61ms/step - loss: 2.0355 - accuracy: 0.2415 - val_loss: 2.3962 - val_accuracy: 0.1102
Epoch 2/20
195/195 [==============================] - 11s 54ms/step - loss: 1.7014 - accuracy: 0.3752 - val_loss: 2.4377 - val_accuracy: 0.1581
Epoch 3/20
195/195 [==============================] - 10s 51ms/step - loss: 1.5245 - accuracy: 0.4478 - val_loss: 1.6395 - val_accuracy: 0.4035
Epoch 4/20
195/195 [==============================] - 10s 53ms/step - loss: 1.3828 - accuracy: 0.5015 - val_loss: 1.5452 - val_accuracy: 0.4429
Epoch 5/20
195/195 [==============================] - 10s 53ms/step - loss: 1.2640 - accuracy: 0.5466 - val_loss: 1.5274 - val_accuracy: 0.4538
Epoch 6/20
195/195 [==============================] - 10s 53ms/step - loss: 1.1547 - accuracy: 0.5877 - val_loss: 1.5517 - val_accuracy: 0.4527
Epoch 7/20
195/195 [==============================] - 10s 54ms/step - loss: 1.0064 - accuracy: 0.6465 - val_loss: 1.5969 - val_accuracy: